In [ ]:
%%time
import pandas as pd
df = pd.read_csv("drive/My Drive/Veri Setleri/CC_DuyguAnalizi.csv", delimiter = "~", encoding='latin-1')

replaceDict={'Ý':'İ','Ð':'Ğ','Þ':'Ş','þ':'ş','ý':'ı','ð':'ğ'}

for x in replaceDict:
    df=df.replace(x,replaceDict[x],regex=True)
df['TICKET_DESC'] = df['TICKET_DESC'].astype(str)
df.head()

In [ ]:
df_text=df.copy()

# remove emoticon

In [1]:
import re
def replaceemoticon(word):
    pos = re.findall(r"(?::\) | :-\) | =\) | :D | :d | <3 | \(: | :\'\) | \^\^|;\) | \(-:)", word)
    neg = re.findall(r"(:-\( | :\( | ;\( | ;-\( | =\( | :/ | :\\ | -_- | \): | \)-:)", word)
  
    if pos:
        #word = ":)"
        word = word + " pozitif"

    elif neg:
        #word = ":("
        word = word + " negatif"

    return word

In [ ]:
df_text["preProcessing"] = df_text["TICKET_DESC"].map(replaceemoticon)

# remove http, www, .com, email adress, link, punc and digit

In [2]:
from string import punctuation, digits

def preProcessing(text):
  return df_remove_punc(df_remove_digits(unlem(df_remove_email_adressed(df_remove_com_tr(df_remove_link(text))))))

def df_remove_link(text):
  return re.sub(r"((http\S+)|(www\S+))", "", text)

def df_remove_com_tr(text):
    text = re.sub(r'\S*\.(com|tr)',"",text)
    return text

def df_remove_email_adressed(text):
  return re.sub(r'([.\w]{3,}@[.\w]{5,})', '', text)

def df_remove_digits(text):
    text=text.strip()
    remove_digits = str.maketrans('', '', digits)
    return text.translate(remove_digits)

def unlem(text):
    if "!" in text:
      return text+"ünlem"
    else:
      return text

def df_remove_punc(text):
    regex = re.compile('[%s]' % re.escape(punctuation))
    return regex.sub('', text)



In [ ]:
%%time
df_text["preProcessing"] = df_text["preProcessing"].map(preProcessing)

# Bazı cevap ve yorumları çıkarma

In [ ]:
data=df_text.copy()

In [ ]:
import pandas as pd
import numpy as np
import re
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
pd.set_option("display.max_rows", data.shape[0])
pd.set_option('display.max_colwidth', data.shape[0])

In [ ]:
# TICKET_DESC to lower characters
data["preProcessing"] = data["preProcessing"].str.lower()

In [ ]:
#Drop all dupplicates in the TICKET_DESC column
data.drop_duplicates(subset=['preProcessing'], inplace = True)

In [ ]:
# Drop the NA data
data.dropna(subset = ["preProcessing"], inplace = True)

In [ ]:
data["preProcessing"] = data["preProcessing"].str.replace('i̇','i')

In [ ]:
# bilgi verildi geçen verileri çıkartalım 
data = data[~data.preProcessing.str.contains(" bilgi verildi")]
data = data[~data.preProcessing.str.contains(" bilgi verildi.")]
data = data[~data.preProcessing.str.contains("bilgi verildi")]
data = data[~data.preProcessing.str.contains("bilgi verildi.")]
data = data[~data.preProcessing.str.contains("bilgi veirldi")]
data = data[~data.preProcessing.str.contains("bilgi veirldi.")]
# bv geçen verileri çıkartalım 
data = data[~data.preProcessing.str.contains("bv")]
data = data[~data.preProcessing.str.contains("b.v")]
data = data[~data.preProcessing.str.contains("b.v.")]
# bunlar toplamda ~25.000

In [ ]:
#bilgi verildi veya verilmiştir türevleri
bilgi =  (r"(\s*)b.?lg.?s?.?(\s*)?v.?r.?l(d.?|m.?.?t.?r).?")  #14834 tane
#bv ve türevleri
bilgi_kısa =(r".?b.?v.?")

In [ ]:
data = data[~data.preProcessing.str.contains(bilgi)]
data = data[~data.preProcessing.str.contains(bilgi_kısa)]
# toplamda -7500

In [ ]:
# nolu başvuru türevleri
nolu_basvuru = (r".?n.?l.?(\s*)?b.?(s|ş)v.?r.?")            #2592 tane
numaralı_basvuru = (r".?n.?m.?r.?l.?(\s*)?b.?(s|ş)v.?r.?")  #3809 tane

In [ ]:
# nolu başvuru geçen verileri çıkartalım 
data = data[~data.preProcessing.str.contains(nolu_basvuru)]
data = data[~data.preProcessing.str.contains(numaralı_basvuru)]
# toplamda -7000

In [ ]:
# tespit edildi veya edilmiştir türevleri
tespit = (r".?t.?sp.?t(\s*)?.?d.?l(d.?|m.?.?t.?r).?") 

In [ ]:
# tespit edildi geçen verileri çıkartalım 
data = data[~data.preProcessing.str.contains(tespit)]

In [ ]:
# kayıt alın(ama)dı/açıl(ama)dı/alın(ama)mıştır 
kayıt =  (r".?k.?y.?t(\s*)?.?l.?na?(m.?)?(d.?|m.?.?t.?r).?") 
kayıt2 =  (r".?k.?y.?t(\s*)?.?(c|ç).?la?(m.?)?(d.?|m.?.?t.?r).?")

In [ ]:
# kayıt alın(ma)dı veya alınmıştır geçen verileri çıkartalım 
data = data[~data.preProcessing.str.contains(kayıt)]
data = data[~data.preProcessing.str.contains(kayıt2)] 

In [ ]:
# talep alındı ve türevleri
talep =  (r"t.?.?p(\s*)?.?l.?na?(m.?)?(d.?|m.?.?t.?r).?") 
talep2 =  (r"t.?l.?p(\s*)?.?l.?(s|ş)t.?r.?la?(m.?)?(d.?|m.?.?t.?r).?") 

In [ ]:
# talep alın(ma)dı veya alınmıştır geçen verileri çıkartalım 
data = data[~data.preProcessing.str.contains(talep)]
data = data[~data.preProcessing.str.contains(talep2)] 

In [ ]:
# her satırdaki mesajın kelime sayısını 'totalwords' sütununa yaz
data['totalwords'] = data['preProcessing'].str.split().str.len()

In [ ]:
# 7 ile 60 arasında kelime sayısına sahip olan verileri al
data = data[data.totalwords > 6]
data = data[data.totalwords < 61]

In [ ]:
data["totalwords"].mean()

In [ ]:
data["totalwords"].median()

In [ ]:
data["totalwords"].std()

## Tokenize

In [1]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

In [ ]:
data['preProcessing'] = data.apply(lambda row: word_tokenize(row['preProcessing']), axis=1)

## Tokenize to str

In [ ]:
def tokenizeList_to_str(text):
  return " ".join(text)

data['preProcessing'] = data['preProcessing'].map(tokenizeList_to_str)

## Bilinmeyen karakteri sil

In [ ]:
karakterler=[]
def find(text):
  for i in text:
    if(i not in ["q","x","w","a","b","c","ç","d","e","f","g","ğ","h","ı","i","j","k","l","m","n","o","ö","p","r","s","ş","t","u","ü","v","y","z"]):
      karakterler.append(i)
    else:
      continue
data["preProcessing"].map(find)


In [ ]:
silinecek_unique = set(karakterler)
for i in silinecek_unique:
  data["preProcessing"]=data["preProcessing"].str.replace(i," ")

## Tek harfleri sil

In [ ]:
def sil_1(text):
    text_list=text.split(" ")
    for i in text_list:
        if len(i)==1:
            text_list.remove(i)   
    return " ".join(text_list)

data["preProcessing"]=data["preProcessing"].map(sil_1)

## Stopwords sil

In [ ]:
WPT = nltk.WordPunctTokenizer()
stop_word_list = nltk.corpus.stopwords.words('turkish')
stop_word_list.remove('ama')





In [ ]:
def stopwords_sil(text):
    text_list=text.split(" ")
    silinmis_list = [text for text in text_list if text not in stop_word_list]
    silinmis_list=" ".join(silinmis_list)
    ##print(silinmis_list)
    return silinmis_list

In [ ]:
data["preProcessing"]=data["preProcessing"].map(stopwords_sil)

## Lang detect (ing satırları sil)

In [ ]:
#pip install fasttext

In [ ]:
import fasttext
PRETRAINED_MODEL_PATH = '/lid.176.bin'
model = fasttext.load_model(PRETRAINED_MODEL_PATH)

In [ ]:
def ing_sil(text):
    predictions = model.predict([text], k=2)
    if predictions[0][0][0] =="__label__tr":
        y=predictions[1][0][0]
        if y > 0.40:
            return "tr"

In [ ]:
data["lang"]=data["preProcessing"].map(ing)

In [ ]:
data_ingsiz=data.dropna(subset=['lang'])